In [78]:
from __future__ import annotations
import numpy as np
import matplotlib.pyplot as plt
import random
import time
from typing import List
from statistics import mean
from dataclasses import dataclass
from pygame_visualization import VRPVisualizator
from copy import deepcopy

In [79]:
GRAPH_SIZE = 5
X_MIN = -600
X_MAX = 600
Y_MIN = -400
Y_MAX = 400
MIN_DISTANCE_BETWEEN_POINTS = 0.4
VIZU_SCALING = 0.5
K_TRUCKS = 1
K_MEANS_SLEEP = 0

In [80]:
vizu = VRPVisualizator()
vizu.set_scaling(VIZU_SCALING)

In [81]:
def manhattan(a, b):
    return sum(abs(val1-val2) for val1, val2 in zip(a,b))

In [82]:
def euclidian(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.linalg.norm(a-b)

In [83]:
random.seed(42)
def generate_coordinates(n_nodes: int, x_coords_range, y_coords_range, min_distance: float):
    generated_coords = [(0, 0)]
    vizu.add_node(0, 0)
    for i in range(n_nodes - 1):
        is_ok = False
        while not is_ok:
            coord = (random.uniform(*x_coords_range), random.uniform(*y_coords_range))
            is_ok = True
            for node in generated_coords:
                if euclidian(coord, node) < min_distance:
                    is_ok = False
        generated_coords.append(coord)
        vizu.add_node(coord[0], coord[1])
    return generated_coords

In [84]:
points = generate_coordinates(GRAPH_SIZE, (X_MIN, X_MAX), (Y_MIN, Y_MAX), MIN_DISTANCE_BETWEEN_POINTS)
random.seed()

# K-means clustering

## 1- k-means++ centroids selection

In [85]:
def select_initial_centroids(points: list, k: int, display) -> list:
    vizu.clear_centroids_and_color()
    #Initialize list of centroids with a first random chosen one.
    first_centroid = list(random.choice(points))
    vizu.add_centroid(first_centroid)
    centroids = [first_centroid]
    for i_centroid in range(k - 1):
        print('GENERATED:', i_centroid)
        #Store minimum distances of each points to a centroid.
        all_distances = []
        for i_point, point in enumerate(points):
            min_distance = float('inf')
            best_i_centroid = 0
            #Compute the lowest distance to a previously selected
            #centroid from this point.
            for i_centroid in range(len(centroids)):
                distance = manhattan(point, centroids[i_centroid])
                if distance < min_distance:
                    min_distance = distance
                    best_i_centroid = i_centroid

            vizu.bind_to_centroid(i_point, best_i_centroid)
            if display:
                vizu.add_centroid_line(point, centroids[best_i_centroid])
                time.sleep(K_MEANS_SLEEP)
            all_distances.append(min_distance)
        
        #Select the point with maximum distance as our new centroid
        next_centroid = list(points[np.argmax(np.array(all_distances))])
        centroids.append(next_centroid)
        vizu.add_centroid(next_centroid)
        vizu.clear_centroid_lines()
    return centroids

In [86]:
@dataclass
class SubPoint():
    global_index: int
    coordinates: tuple

def calc_centroid(points):
    points = np.array(points)
    length = points.shape[0]
    sum_x = np.sum(points[:, 0])
    sum_y = np.sum(points[:, 1])
    return [sum_x/length, sum_y/length]

def generate_clusters(points, centroids, display) -> List[List[SubPoint]]:
    centroids_childs = dict()
    centroid_changed = True

    while centroid_changed:
        for i in range(len(centroids)):
            centroids_childs[i] = []

        for i_point, point in enumerate(points):
            #Get the closest centroid
            min_distance = float('inf')
            best_i_centroid = 0
            for i_centroid in range(len(centroids)):
                distance = manhattan(point, centroids[i_centroid])
                if distance < min_distance:
                    best_i_centroid = i_centroid
                    min_distance = distance

            #Create subpoint to keep track of the global index of the node
            #Global index is i+1 because i = 0 = depot
            subpoint = SubPoint(i_point + 1, point)
            centroids_childs[best_i_centroid].append(subpoint)
            vizu.bind_to_centroid(i_point, best_i_centroid)
            if display:
                vizu.add_centroid_line(point, centroids[best_i_centroid])
                time.sleep(K_MEANS_SLEEP)

        if display:
            vizu.clear_centroid_lines()
        centroid_changed = False
        #Update centroids values
        for i_centroid, centroid in enumerate(centroids):
            childs_coordinates = [subpoint.coordinates for subpoint in centroids_childs[i_centroid]]
            new_centroid_value = calc_centroid(childs_coordinates)
            if new_centroid_value != centroid:
                centroid_changed = True
                centroid[0] = new_centroid_value[0]
                centroid[1] = new_centroid_value[1]
    vizu.clear_centroids()
    clusters = [centroids_childs[centroid] for centroid in centroids_childs.keys()]
    return clusters

In [87]:
def generate_distance_matrix(points: list):
    length = len(points)
    matrix = np.empty((length, length))
    matrix[:] = np.nan
    for i in range(length):
        for j in range(length):
            if i != j:
                dist = round(euclidian(points[i], points[j]), 2)
                matrix[i][j] = dist
                matrix[j][i] = dist
    return matrix

def generate_graphs_from_clusters(clusters):
    graphs = []
    for cluster in clusters:
        #Start by adding the depot point
        points_coordinates = [points[0]]
        points_indexes = [0]
        for subpoint in cluster:
            points_coordinates.append(subpoint.coordinates)
            points_indexes.append(subpoint.global_index)
        distance_matrix = generate_distance_matrix(points_coordinates)
        graphs.append((points_indexes, distance_matrix))
    return graphs

In [88]:
points_without_0 = list(points)
points_without_0.pop(0)
centroids = select_initial_centroids(points_without_0, K_TRUCKS, False)
clusters = generate_clusters(points_without_0, centroids, False)
graphs = generate_graphs_from_clusters(clusters)
vizu.draw()
print(graphs[0])

([0, 1, 2, 3, 4], array([[   nan, 415.19, 349.16, 317.03, 575.04],
       [415.19,    nan, 465.14, 534.2 , 307.32],
       [349.16, 465.14,    nan, 661.99, 748.56],
       [317.03, 534.2 , 661.99,    nan, 507.46],
       [575.04, 307.32, 748.56, 507.46,    nan]]))


In [89]:
class ALGO_neightbours():
    def __init__(self, graph, start: int) -> None:
        self.graph = deepcopy(graph)
        self.start = start
        self.best_length = 0
        self.current_node = 0
        self.path_history = [0]

    def run(self) -> list:   
        while len(self.path_history) < (len(self.graph)):
            actual_graph = self.graph[self.current_node]
            for i in self.path_history:
                actual_graph[i] = float("inf")
            min_distance = np.nanmin(actual_graph)
            next_node = np.where(actual_graph == min_distance)[0][0]
            self.path_history.append(next_node)
            self.current_node = next_node
            self.best_length += min_distance
        
        self.path_history.append(0)
        self.best_length += self.graph[self.current_node][0]
        return self.best_length,self.path_history    

In [90]:
#Evaporation factor for global update of pheromones
RHO = 0.1
#Evaporation factor for local update of pheromones
KAPPA = RHO
#Q
OMICRON = 1
#Impact of pheromones
ALPHA = 1
#Impact of weights
BETA = 2
#Initial pheromones
TAU = 1
#Exploration/exploitation trade off
EPSILON = 0.9

In [91]:
class Ant():
    def __init__(self, graph, pheromones, real_nodes_indexes, initial_tau, init_pos: int, vizualizator: VRPVisualizator) -> None:
        self.graph = graph
        self.pheromones = pheromones
        self.nodes_amount = len(graph[0])
        self.real_nodes_indexes = real_nodes_indexes
        self.initial_tau = initial_tau
        self.init_pos = init_pos
        self.vizualizator = vizualizator
        self.current_pos = init_pos
        self.path_history = [init_pos]
        self.unvisited_nodes = dict()
        self.distance = 0
        self.can_continue = True
        self.init_unvisited_nodes()

    def init_unvisited_nodes(self):
        for i in range(self.nodes_amount):
            if i != self.init_pos:
                self.unvisited_nodes[i] = 0

    def add_to_path_history(self, node) -> None:
        self.path_history.append(node)
        self.unvisited_nodes.pop(node, None)
        if self.vizualizator:
            self.vizualizator.set_path(self.get_real_path(self.path_history))

    def move(self) -> None:
        #Return to the start if there is no destination available
        if not self.unvisited_nodes:
            dest_picked = self.init_pos
            self.can_continue = False
        else:
            dest_picked = self.pick_dest()

        self.distance += self.graph[self.current_pos][dest_picked]
        self.update_pheromones_locally(dest_picked)
        self.current_pos = dest_picked
        self.add_to_path_history(self.current_pos)

    def pick_dest(self) -> int:
        best_viability = float('-inf')
        best_dest = None
        #calculate possibles destinations weights
        for node in self.unvisited_nodes:
            weight = self.pheromones[self.current_pos][node] * (1/self.graph[self.current_pos][node])**BETA
            if weight > best_viability:
                best_viability = weight
                best_dest = node
            self.unvisited_nodes[node] = weight

        choice = random.uniform(0, 1)

        if choice <= EPSILON:
            return best_dest
        else:
            return self.biased_exploration()

    def biased_exploration(self) -> int:
        probabilities = []
        denominator = 0
        
        #Calculate the denominator first
        total_weights = sum(self.unvisited_nodes.values())

        #Calculate all probabilities
        for node in self.unvisited_nodes:
            self.unvisited_nodes[node] /= total_weights
        
        #Calculate cumulative sum
        cumulative = 0
        for node in self.unvisited_nodes:
            cumulative += self.unvisited_nodes[node]
            self.unvisited_nodes[node] = cumulative

        rand = random.uniform(0, 1)
        for node in self.unvisited_nodes:
            if self.unvisited_nodes[node] >= rand:
                return node

    def update_pheromones_locally(self, dest) -> None:
        #Apply the ACS local updating rule
        #evaporate
        new_value = self.pheromones[self.current_pos][dest] * (1 - RHO)
        self.pheromones[self.current_pos][dest] = max(new_value, self.initial_tau)
        self.pheromones[dest][self.current_pos] = max(new_value, self.initial_tau)
        
        self.pheromones[self.current_pos][dest] += RHO * self.initial_tau
        self.pheromones[dest][self.current_pos] += RHO * self.initial_tau
    
    def get_real_node_index(self, index):
        return self.real_nodes_indexes[index]

    def get_real_path(self, path):
        real_path = []
        for n in path:
            real_path.append(self.get_real_node_index(n))
        return real_path

In [92]:
class ACO():
    def __init__(self, graph, real_nodes_indexes, start: int, vizualizator: VRPVisualizator = None) -> None:
        self.graph = graph
        self.real_nodes_indexes = real_nodes_indexes
        self.start = start
        self.vizualizator = vizualizator
        self.current_best_path = None
        self.current_best_length =  float('inf')
        algo_neighbOURS = ALGO_neightbours(graph, 0)
        length, path = algo_neighbOURS.run()
        self.initial_tau = (length*30)**-1
        #print('TAU:', self.initial_tau)
        self.pheromones = np.full(graph.shape, self.initial_tau)

    def run(self, iter: int) -> list:
        for i in range(iter):
            #print('Tour:', i)
            self.tour_construction(10)
            self.global_update_pheromones()
        real_best_path = self.get_real_path(self.current_best_path)
        if self.vizualizator:
            self.vizualizator.add_path(real_best_path)
            self.vizualizator.clear_current_path()
        #print('Best:', real_best_path)
        #print('Length:', self.current_best_length)
        return real_best_path, self.current_best_length

    def tour_construction(self, ant_amount: int) -> None:
        ants = [Ant(self.graph, self.pheromones, self.real_nodes_indexes, self.initial_tau, random.randint(0, len(self.real_nodes_indexes)-1), self.vizualizator) for i in range(ant_amount)]
        while ants:
            for ant in ants:
                if ant.can_continue:
                    ant.move()
                else:
                    self.update_current_best(ant)
                    ants.remove(ant)

    def update_current_best(self, ant):
        ant.distance = round(ant.distance, 2)
        if ant.distance < self.current_best_length:
            #print('NEW BEST:', path)
            print('BEST LENGTH:', ant.distance)
            self.current_best_path = ant.path_history
            self.current_best_length = ant.distance

    def global_update_pheromones(self) -> None:
        length = len(self.pheromones[0])
        #Evaporation
        for i in range(length):
            for j in range(length):
                new_value = self.pheromones[i][j] * (1 - RHO)
                self.pheromones[i][j] = max(new_value, self.initial_tau)

        #New pheromones
        for i in range(len(self.current_best_path)-1):
            current_node = self.current_best_path[i]
            next_node = self.current_best_path[i+1]
            self.pheromones[current_node][next_node] += RHO * (KAPPA / self.current_best_length)
            self.pheromones[next_node][current_node] += RHO * (KAPPA / self.current_best_length)
    
    def get_real_node_index(self, index):
        return self.real_nodes_indexes[index]

    def get_real_path(self, path):
        real_path = []
        for n in path:
            real_path.append(self.get_real_node_index(n))
        return real_path

In [93]:
def get_dataset_graph(dataset):
    points = []
    points_indices = []
    with open(dataset+'.txt', 'r') as file:
        i = 0
        for line in file.readlines():
            x, y = line.split(',')
            x = float(x)
            y = float(y)
            points.append((float(x), float(y)))
            points_indices.append(i)
            i += 1
            vizu.add_node(float(x), float(y))
    return points_indices, generate_distance_matrix(points)

In [94]:
def oliver_30():
    vizu = VRPVisualizator()
    oliver30 = get_dataset_graph('oliver30')
    vizu.clear_paths()
    alls_dists = []
    start = time.time()
    for i in range(1):
        print('iter:', i)
        aco = ACO(oliver30[1], oliver30[0], 0)
        path, distance = aco.run(2500)
        alls_dists.append(distance)
    print('TOTAL TIME:', time.strftime('%H:%M:%S', time.gmtime(time.time()-start)))
    #vizu.add_path(path)
    print('MEAN:', mean(alls_dists))

In [95]:
oliver_30()

: 657
Tour: 658
Tour: 659
Tour: 660
Tour: 661
Tour: 662
Tour: 663
Tour: 664
Tour: 665
Tour: 666
Tour: 667
Tour: 668
Tour: 669
Tour: 670
Tour: 671
Tour: 672
Tour: 673
Tour: 674
Tour: 675
Tour: 676
Tour: 677
Tour: 678
Tour: 679
Tour: 680
Tour: 681
Tour: 682
Tour: 683
Tour: 684
Tour: 685
Tour: 686
Tour: 687
Tour: 688
Tour: 689
Tour: 690
Tour: 691
Tour: 692
Tour: 693
Tour: 694
Tour: 695
Tour: 696
Tour: 697
Tour: 698
Tour: 699
Tour: 700
Tour: 701
Tour: 702
Tour: 703
Tour: 704
Tour: 705
Tour: 706
Tour: 707
Tour: 708
Tour: 709
Tour: 710
Tour: 711
Tour: 712
Tour: 713
Tour: 714
Tour: 715
Tour: 716
Tour: 717
Tour: 718
Tour: 719
Tour: 720
Tour: 721
Tour: 722
Tour: 723
Tour: 724
Tour: 725
Tour: 726
Tour: 727
Tour: 728
Tour: 729
Tour: 730
Tour: 731
Tour: 732
Tour: 733
Tour: 734
Tour: 735
Tour: 736
Tour: 737
Tour: 738
Tour: 739
Tour: 740
Tour: 741
Tour: 742
Tour: 743
Tour: 744
Tour: 745
Tour: 746
Tour: 747
Tour: 748
Tour: 749
Tour: 750
Tour: 751
Tour: 752
Tour: 753
Tour: 754
Tour: 755
Tour: 756
Tour

In [96]:
vizu.clear_paths()
result_paths = []
result_distances = []
start = time.time()
for graph in graphs:
    aco = ACO(graph[1], graph[0], 0)
    path, distance = aco.run(50)
    vizu.add_path(path)
    result_paths.append(path)
    result_distances.append(distance)
print('TOTAL TIME:', time.strftime('%H:%M:%S', time.gmtime(time.time()-start)))


Tour: 0
BEST LENGTH: 1946.11
Tour: 1
Tour: 2
Tour: 3
Tour: 4
Tour: 5
Tour: 6
Tour: 7
Tour: 8
Tour: 9
Tour: 10
Tour: 11
Tour: 12
Tour: 13
Tour: 14
Tour: 15
Tour: 16
Tour: 17
Tour: 18
Tour: 19
Tour: 20
Tour: 21
Tour: 22
Tour: 23
Tour: 24
Tour: 25
Tour: 26
Tour: 27
Tour: 28
Tour: 29
Tour: 30
Tour: 31
Tour: 32
Tour: 33
Tour: 34
Tour: 35
Tour: 36
Tour: 37
Tour: 38
Tour: 39
Tour: 40
Tour: 41
Tour: 42
Tour: 43
Tour: 44
Tour: 45
Tour: 46
Tour: 47
Tour: 48
Tour: 49
TOTAL TIME: 00:00:00


In [97]:
print('TOTAL DISTANCE:', sum(result_distances))
print('PATHS:', result_paths)

TOTAL DISTANCE: 1946.11
PATHS: [[1, 4, 3, 0, 2, 1]]


In [98]:
vizu.clear_paths()
for path in result_paths:
    vizu.add_path(path)